In [2]:
%pip install -Uq chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 50.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 83.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 7.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 7

# 데이터 불러오기


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
import json
json_directory = "/content/drive/MyDrive/hidoc_qna/json"
data = []

for filename in os.listdir(json_directory):
    if filename.endswith(".json"):
        file_path = os.path.join(json_directory, filename)
        with open(file_path, "r") as file:
            json_data = json.load(file)
            json_data['question_id'] = filename.split("_")[-1].split(".")[0]
            data.append(json_data)

In [4]:
from chromadb.utils import embedding_functions
default_ef = embedding_functions.DefaultEmbeddingFunction()

In [12]:
import chromadb
# 크로마 DB 클라이언트 생성
chroma_client = chromadb.Client()
collection = chroma_client.create_collection(name="question_collection",embedding_function=default_ef)

In [6]:
chroma_client.delete_collection(name="question_collection")

In [7]:
len(data)

1772

In [8]:
questions = []
question_ids = []
question_metadatas =[]
check_duplicate = set()
for item in data:
    question_text = item['question']['content']
    question_id = item['question_id']
    if question_id not in check_duplicate:
      questions.append(question_text)
      question_ids.append(question_id)
      check_duplicate.add(question_id)
      question_metadata = {
        'title': item['question']['header']['title'],
        'date': item['question']['header']['date'],
        'tags': ','.join(item['question']['header']['smart_tags'])
      }
      question_metadatas.append(question_metadata)

In [9]:
questions[1], question_ids[1], question_metadatas[1]

('엉덩이 꼬리뼈맞나 암튼 거기쪽에 한 1년전쯤부터있었던것같은데 커진지 작아진건지 잘모르겠거든요 근데 커진것같아요 요새 가만히 있어도 허리아프던데 이것때문인가해서요 약간 허리가 약해진 느낌이랄까 학생이라 앉아있는 시간이 많거든요 한 1주전인가 그쯤에 움직일때 엄청아프던데 지금은 안그래요 암튼 모양은 길죽하고 저번에 꽉눌렀을때 약간 위로 움직인것같은데 기분탓일수도 있어요 암튼 꽤나 오래됐고 걍 신경 안쓰며 살았는데 유튜브에서 골육종 연육종인가 그런게 있다길래 걱정되네요 겉으론 안보이고 눌러도 안아픕니다 그냥 애초에 꼬리뼈에 물렁뼈가 있나요? 그건아닐것같은데 육종암이 골반 척추라는데 꼬리뼈에도 저처럼 가능한가요 그리고 앞서말한 증상위험한건가요?신경쓰이네요 움직일때 막 아프거나 그러진 않아요 아 그리고 오랜만에 몸무게 측정해보니 3키로 가량 빠져있는데 원래잘 안빠지거든요 이것도 살짝 걱정이 되네요 근데 학교에서 공복상태로 몸무게 측정했을때 딱 그러긴했어요',
 'C0000886158',
 {'title': '엉덩이 꼬리뼈 멍울', 'date': '2024.05.06', 'tags': '항문,뼈,외과'})

In [10]:
len(questions), len(question_ids)

(1772, 1772)

In [13]:
collection.add(
    documents=questions[:100],
    ids=question_ids[:100],
    metadatas=question_metadatas[:100],
)

In [14]:
results = collection.query(
    query_texts='어깨가 너무 자주 아파요',
    n_results=1,
)
results

{'ids': [['C0000886058']],
 'distances': [[0.4607261121273041]],
 'metadatas': [[{'date': '2024.05.05',
    'tags': '뼈,발목,정형외과,통증',
    'title': '발목 뼈 소리 후 통증'}]],
 'embeddings': None,
 'documents': [['잘못된 자세로 있다 발목 뼈 소리 후 통증이 생겼어요 이후 뼈소리도 좀 나네요 ? 왜 그런거죠 ㅠㅠㅠㅠㅠ']],
 'uris': None,
 'data': None}

In [15]:
results = collection.query(
    query_texts='어깨가 너무 자주 아파요',
    n_results=1,
)

for i, result in enumerate(results['documents'][0]):
    question_id = results['ids'][0][i]
    question_metadata = results['metadatas'][0][i]

    print(f"유사한 질문 {i+1}: {result}")
    print(f"질문 ID: {question_id}")
    print(f"메타데이터:")
    print(f"  제목: {question_metadata['title']}")
    print(f"  날짜: {question_metadata['date']}")
    print(f"  태그: {question_metadata['tags']}")

    # 원본 데이터에서 해당 질문의 답변 검색
    for item in data:
        if item['question_id'] == question_id:
            answers = item['answers']
            print(f"답변:")
            for answer in answers:
                answer_cnt = ''.join(answer['content'].split('\n')[1:])
                print(f"  - {answer_cnt}")
            break

    print("---")


유사한 질문 1: 잘못된 자세로 있다 발목 뼈 소리 후 통증이 생겼어요 이후 뼈소리도 좀 나네요 ? 왜 그런거죠 ㅠㅠㅠㅠㅠ
질문 ID: C0000886058
메타데이터:
  제목: 발목 뼈 소리 후 통증
  날짜: 2024.05.05
  태그: 뼈,발목,정형외과,통증
답변:
  - 골절보다는 힘줄의 문제로 소리가 나는 경우가 많으며인대가 파열되거나 하는 경우는 드뭅니다.통증이 오랜 기간 지속되면 인대 손상의 가능성도 있습니다.
  - 발목에서 소리가 나면서 이후 통증이 발생한 상태로 이해됩니다. 불편한 자세에서 발생한 증상이니 근육이나 인대의 부상부터 의심해보아야 하겠습니다. 통증이 지속되거나 부어오르고 붉어지는 등의 변화를 보이면 정형외과를 방문해 진료받으시기 바랍니다.
---


In [17]:
# 유사한 질문 검색
search_results = collection.query(
    query_texts="어깨가 너무 자주 아파요",
    n_results=5,
)

docs_string = ""
user_question = "어깨가 너무 아파요"
for i, result in enumerate(search_results['documents'][0]):
    question_id = search_results['ids'][0][i]
    # 원본 데이터에서 해당 질문의 답변 검색
    for item in data:
        if item['question_id'] == question_id:
            answers = item['answers']
            for answer in answers:
                answer_cnt = ''.join(answer['content'].split('\n')[1:])
                docs_string += answer_cnt + "\n\n"
            break

# 메타데이터의 태그를 활용하여 전문의 정보 생성
tags = search_results['metadatas'][0][0]['tags']
expert_info = ""
for tag in tags.split(','):
    expert_info += f"{tag} 전문의\n"

# 프롬프트 생성
prompt = f"""
[DOCS]
{docs_string}

[EXPERT INFO]
{expert_info}

[INSTRUCTION]
위의 [DOCS]에 있는 문서 내용을 참조하여 아래 사용자의 질문에 답변해주세요. 답변은 [EXPERT INFO]에 명시된 전문의 관점에서 제공해주세요.

---

user: {user_question}
assistant:
"""
print(prompt)


[DOCS]
골절보다는 힘줄의 문제로 소리가 나는 경우가 많으며인대가 파열되거나 하는 경우는 드뭅니다.통증이 오랜 기간 지속되면 인대 손상의 가능성도 있습니다.

발목에서 소리가 나면서 이후 통증이 발생한 상태로 이해됩니다. 불편한 자세에서 발생한 증상이니 근육이나 인대의 부상부터 의심해보아야 하겠습니다. 통증이 지속되거나 부어오르고 붉어지는 등의 변화를 보이면 정형외과를 방문해 진료받으시기 바랍니다.

전신 두드러기가 심한 상황이라 추정됩니다.일차적으로는 피부과가 가장 적합한 진료과이고, 타 진료과의 추가진료가 필요할 시 진료본 의사가 진료의뢰서를 작성해줄 것이라 예상됩니다. 우선 피부과를 방문해보시기 바랍니다.

배를 꾹 누르면 콩닥콩닥 하는 소리는 심장 박동이 아니라 장기의 움직임입니다. 장기는 끊임없이 움직이면서 소화, 배설, 호흡 등 다양한 활동을 합니다. 따라서 배를 누르면 장기의 움직임이 느껴질 수 있습니다. 이는 정상적인 현상이므로 걱정하지 않으셔도 됩니다.

배를 눌렀을 때 두근거리는 느낌을 받으시는 부위가 몸의 정중앙에 가깝다면 이는 복부대동맥의 박동을 느끼는 것이라 추정할 수 있습니다. 물론 마른 경우에만 느껴지고, 지방이 어느정도 쌓이면 박동을 느끼기는 어렵습니다. 또, 손끝의 박동을 배에서 느껴지는 박동이라 오해하는 경우도 있습니다.

복부 장기의 움직임은 일반적으로는 느껴지지 않습니다. 만약 심장이 뛰는 것처럼 느껴진다면 복부 대동맥과 같은 혈관의 박동이 아닌지 확인이 필요합니다.

너무 힘드시겠네요. 그래도  식욕이 있으시다는것은 없는것보다는 좋은 것 같습니다. 소화 잘되는 것을 찾아서 소량씩  드셔보셔요. 밀가루 우유  커피 이런것은  소화가 잘안될수 있어요

상기증상이 지속된다면 위염 식도염 위궤양과 같은 위장관 질환과 담낭 질환에 대한 평가가필요하므로 위내시경과 복부 초음파 검사가 도움이 되니 소화기 내과 진료를 보실 것을 권해드립니다.답변이 도움이 되었기를 바랍니다.

소화불량이 지속적으로 나타난다면 소화기관에 질환이 있어서

In [18]:
import requests

In [19]:
datas = {
    "model" : "gemma:7b",
    "messages" : [
        {
            "role" : "user",
            "content" : prompt
        }
    ],
    "stream" : False
}

In [20]:
res = requests.post('http://koock.iptime.org:11434/api/chat',json=datas, stream=False)

In [21]:
import json
response_dict = json.loads(res.text)
content = response_dict["message"]["content"]
print(content)

**답변:**

어깨가 너무 아파는 증상은 다양한 원인으로 발생할 수 있습니다. 위의 문서에서 언급된 내용에 따라 가능한 원인 중 하나는 인대 손상일 수 있습니다. 통증이 지속되거나 부어오르고 붉어지는 등의 변화를 보이면 정형외과 전문의를 방문하여 진료받는 것이 좋습니다.

**전문의 추천:**

* 정형외과 전문의를 방문하여 인대 손상 가능성을 확인하고 치료를 받습니다.
* 통증 완화를 위해 물리치료나 약물 치료 등의 관리도 필요합니다.

**참고:**

* 위의 문서에 언급된 내용은 일반적인 정보이며, 개인의 건강 상태 및 질병의 정확한 진단은 전문의에게 상담해야 합니다.
